# Life Cycle Assessment

BioSTEAM facilitates life cycle assessment through a minimalistic interface with stream and utility objects. For demonstration purposes only, a simplified life cycle assessment is performed here for a sugarcane biorefinery assumming that materials other than sugarcane have negligible impacts. Our goal is to get the green house gas emission per kg ethanol along a cradle-to-gate system boundary.   

Assume the following inventory:

|             | GWP 100yr | Reference                                              |
| ----------- | --------- | ------------------------------------------------------ |
| Sugarcane   | 0.46     | https://www.osti.gov/servlets/purl/1337146             |
| Electricity | 0.480     | [https://greet.es.anl.gov/](https://greet.es.anl.gov/) |


In [12]:
import biosteam as bst
from biorefineries import sugarcane as sc

sc.load()
sys = sc.sugarcane_sys
feeds = sys.feeds
material_cradle_to_gate_key = ('GWP100', 'cradle-to-gate')
coproduct_gate_to_grave = ('GWP100', 'gate-to-grave')
electricity_key = ('GWP100', 'cradle-to-gate')

for feed in sys.feeds:
    # Assume all other feeds are negligible
    feed.characterization_factors[material_cradle_to_gate_key] = 0.

# Set non-negligible characterization factors
dw = 1. - sc.sugarcane.get_mass_composition('Water')
sc.sugarcane.characterization_factors[material_cradle_to_gate_key] = 0.46 * dw # https://www.osti.gov/servlets/purl/1337146
bst.PowerUtility.characterization_factors[electricity_key] = 0.48 # https://greet.es.anl.gov/
# Assume fertirrigation of vinasse offsets NO2 emissions from fertilizers, so GWP from NO2 does not need to be added.
# However, the high COD leads to CH4 production equivalent to 345 kg–CO2–eq m3 EtOH
# https://doi.org/10.1016/j.biombioe.2007.12.006
sc.vinasse.characterization_factors[coproduct_gate_to_grave] = 345 / 789 * sc.ethanol.F_mass / sc.vinasse.F_mass # kg–CO2–eq kg 
sc.enzyme.characterization_factors[material_cradle_to_gate_key] = 2.24 # General enzyme
sc.H3PO4.characterization_factors[material_cradle_to_gate_key] = 0. # TODO
sc.lime.characterization_factors[material_cradle_to_gate_key] = 1.29

# Get net electricity data
net_electricity = bst.PowerUtility.sum([i.power_utility for i in sys.units])
GWP_material = sum([s.get_impact(material_cradle_to_gate_key) for s in sys.feeds])
biogenic_CO2 = sc.sugarcane.get_atomic_flow('C')*44 # in kg CO2/hr
GWP_direct_emissions = sum([i.imass['CO2'] for i in sys.products if 'CO2' in i.chemicals]) - biogenic_CO2
GWP_electricity_production = net_electricity.get_impact(production_key=electricity_key)
GWP_coproducts = sc.vinasse.get_impact(coproduct_gate_to_grave)
GWP_total = GWP_material + GWP_direct_emissions + GWP_electricity_production + GWP_coproducts # kg CO2 eq. / hr
GWP_sugarcane_ethanol = GWP_total / sc.ethanol.F_mass # kg CO2 eq. / kg sugarcane ethanol
print(GWP_sugarcane_ethanol)

-0.9729999646935572


In [9]:
import biosteam as bst
from biorefineries import corn as cn

cn.load()
sys = cn.corn_sys
feeds = sys.feeds
material_cradle_to_gate_key = ('GWP100', 'cradle-to-gate')
electricity_key = ('GWP100', 'cradle-to-gate')

for feed in sys.feeds:
    # Assume all other feeds are negligible
    feed.characterization_factors[material_cradle_to_gate_key] = 0.

# Set non-negligible characterization factors
dw = 1. - cn.sugarcane.get_mass_composition('Water')
cn.corn.characterization_factors[material_cradle_to_gate_key] = 0.20 * dw # https://wcroc.cfans.umn.edu/research/renewable-energy/lca-corn-grain
bst.PowerUtility.characterization_factors[electricity_key] = 0.48 # https://greet.es.anl.gov/
# Assume fertirrigation of vinasse offsets NO2 emissions from fertilizers, so GWP from NO2 does not need to be added.
# However, the high COD leads to CH4 production equivalent to 345 kg–CO2–eq m3 EtOH
# https://doi.org/10.1016/j.biombioe.2007.12.006
sc.vinasse.characterization_factors[coproduct_gate_to_grave] = 345 / 789 * sc.ethanol.F_mass / sc.vinasse.F_mass # kg–CO2–eq kg 
sc.enzyme.characterization_factors[material_cradle_to_gate_key] = 2.24 # General enzyme
sc.H3PO4.characterization_factors[material_cradle_to_gate_key] = 0. # TODO
sc.lime.characterization_factors[material_cradle_to_gate_key] = 1.29

# Get net electricity data
net_electricity = bst.PowerUtility.sum([i.power_utility for i in sys.units])
GWP_material = sum([s.get_impact(material_cradle_to_gate_key) for s in sys.feeds])
biogenic_CO2 = sc.sugarcane.get_atomic_flow('C')*44 # in kg CO2/hr
GWP_direct_emissions = sum([i.imass['CO2'] for i in sys.products if 'CO2' in i.chemicals]) - biogenic_CO2
GWP_electricity_production = net_electricity.get_impact(production_key=electricity_key)
GWP_coproducts = sc.vinasse.get_impact(coproduct_gate_to_grave)
GWP_total = GWP_material + GWP_direct_emissions + GWP_electricity_production + GWP_coproducts # kg CO2 eq. / hr
GWP_sugarcane_ethanol = GWP_total / sc.ethanol.F_mass # kg CO2 eq. / kg sugarcane ethanol
print(GWP_sugarcane_ethanol)

48666.906437024845

In [11]:
sys.feeds

[<Stream: sugarcane>,
 <Stream: enzyme>,
 <Stream: s417>,
 <Stream: lime>,
 <Stream: s418>,
 <Stream: stripping_water>,
 <Stream: polymer>,
 <Stream: s420>,
 <Stream: H3PO4>,
 <Stream: rvf_wash_water>,
 <Stream: s419>,
 <Stream: denaturant>,
 <Stream: dryer_air>,
 <Stream: cooling_tower_chemicals>,
 <Stream: boiler_makeup_water>,
 <Stream: s413>,
 <Stream: s414>,
 <Stream: cooling_tower_makeup_water>,
 <Stream: dryer_natural_gas>,
 <Stream: natural_gas>,
 <Stream: s.6>,
 <Stream: dilution_water>,
 <Stream: recirculated_chilled_water>,
 <Stream: makeup_water>,
 <Stream: s411>,
 <Stream: s412>,
 <Stream: imbibition_water>]